My dataset is about baseball. My dataset consists of 5 features pertaining to various aspects of 10 baseball games.
The data will be used for learning to predict the win/loss outcome of a team in a baseball game.

Feature 1: Pitcher ERA
Pitcher ERA is the average number of points a pitcher allows the other team to score in a game.
The Pitcher ERA can be any continuous non-negative number. For the sake of this dataset we round the values to 2 decimal palces. 

Feature 2: Scored 3+ Runs
Scored 3+ Runs is a Yes/No value pertaining to whether the team scored at least 3 points in the game.

Feature 3: Batting Practice
Batting Practice is a Yes/No value pertaining to whether the team participated in batting practice before the game.

Feature 4: Pregame Meal
Pregame Meal represents the meal the team ate before the game. It can take on the values Pizza, Pasta, Steak, or Sushi. 

Feature 5: Weather
Weather represents the weather during the game. It can take on values Bad, Fair, or Good.

Class Label: Outcome
The Class Label can take on values Win, or Loss. It represents whether the team won or lost the baseball game.

In [10]:
import pandas as pd

print('\nPart 2:')

#Read dataset.csv
training_df = pd.read_csv("assets/dataset.csv")

#Print the first 4 rows of training_df
print("\n\nThe first 4 rows of training_df:\n")
print(training_df.head(4))

print('\n')

#Print the last 3 rows of training_df
print("The last 3 rows of training_df:\n")
print(training_df.tail(3))

print('\n')

#Print the total number of rows of training_df
num_of_rows = len(training_df.index)
print("Number of rows in training_df:\n" + str(num_of_rows))

print('\n')

#Print the features and class label names
print("Features and class label names:\n")
print(training_df.keys().values)

print('\n')

#For each categorical value, print the possible values it can take
print("\nCategorical values for Scored_3+_Runs:")
print(training_df["Scored_3+_Runs"].unique())

print("\nCategorical values for Batting_Practice:")
print(training_df["Batting_Practice"].unique())

print("\nCategorical values for Pregame_Meal:")
print(training_df["Pregame_Meal"].unique())

print("\nCategorical values for Weather:")
print(training_df["Weather"].unique())

print("\nCategorical values for Outcome:")
print(training_df["Outcome"].unique())


Part 2:


The first 4 rows of training_df:

   Pitcher_ERA Scored_3+_Runs Batting_Practice Pregame_Meal Weather Outcome
0         2.89            Yes              Yes        Pizza    Good     Win
1         4.21             No               No        Pasta    Fair    Loss
2         3.45            Yes               No        Steak     Bad    Loss
3         5.12             No              Yes        Sushi    Good     Win


The last 3 rows of training_df:

   Pitcher_ERA Scored_3+_Runs Batting_Practice Pregame_Meal Weather Outcome
7         4.67            Yes               No        Steak     Bad    Loss
8         3.89             No              Yes        Pasta    Good     Win
9         2.57             No               No        Sushi    Fair    Loss


Number of rows in training_df:
10


Features and class label names:

['Pitcher_ERA' 'Scored_3+_Runs' 'Batting_Practice' 'Pregame_Meal'
 'Weather' 'Outcome']



Categorical values for Scored_3+_Runs:
['Yes' 'No']

Categorical values fo

In [11]:
import math
import pandas as pd


def getData():
    #dataSetName = input("Please enter the dataset file's name:")
    #dataSetName = "assets/" + dataSetName + ".csv"
    dataSetName = "assets/playtennis.csv"
    originalDataSet = pd.read_csv(dataSetName)
    return originalDataSet


#column = Series object
def calculateEntropy(column):    
    numOfValues = column.size # = total number of values in column
    valueCounts = column.value_counts() # = Series object with each unique value and it's number of occurances
    entropy = 0
    for occurances in valueCounts:
        entropy = entropy + logHelper(occurances,numOfValues)
    entropy = entropy * -1
    return entropy
    
    
def logHelper(occurances, numOfValues):
    return occurances/numOfValues * math.log2(occurances/numOfValues)    
    
    
#reducedData is a dataFrame containing the feature column and the target column
def calculate2ndTerm(reducedData, featureColumnName):
    reducedData = reducedData.sort_values(by=featureColumnName) #sort reducedData by feature values to make it easy to iterate
    
    sum = 0 #tracks weighted entropies of partition
    
    countElements = 0.0 #counter needed for iteration
    dataSetSize = reducedData.size / 2 #used to calculate weighting and needed for iteration
    
    prevFeatureValue = "" #used to check for new outcome in sorted reducedData table    
    classValues = [] #used to store target column values for an outcome
    
    tuples = reducedData.itertuples() #get dataFrame as a list of tuples, one tuple for each row
    
    #iterate ruducedData
    for tuple in tuples:
        countElements = countElements + 1
        featureValue = tuple[1]
        
        #if outcome is same as the previous and not the last row in the dataframe, or if it's the first row in the dataframe
        if(countElements != dataSetSize and featureValue == prevFeatureValue or len(classValues) == 0):
            classValue = tuple[2]
            classValues.append(classValue)#get the classValue and add it to the classValues list
            prevFeatureValue = featureValue
            
        #otherwise we've reached a new outcome, or we've reached the last row in the dataframe
        else:
            
            #if outcome is same as previous, and we've reached the last row in the dataframe
            if(featureValue == prevFeatureValue and countElements == dataSetSize):
                classValue = tuple[2]
                classValues.append(classValue)#get the classValue and add it to the classValues list
             
            classValuesDF = pd.DataFrame(classValues)#construct dataframe from classValues list
                        
            outcomeSize = classValuesDF.size#get number of datapoints of outcome
            weighting = outcomeSize / dataSetSize#calculate weighting factor
            
            outcomeEntropy = calculateEntropy(classValuesDF)#calculate entropy of outcome
            weightedEntropy = weighting * outcomeEntropy#apply weighting factor
            sum = sum + weightedEntropy#add new weighted entropy of partition to tracker
            
            classValues = []#reset classValues list for new outcome
            classValue = tuple[2]
            classValues.append(classValue)#get the classValue and add it to the classValues list
            
            prevFeatureValue = featureValue#increment featureValue tracker            
    return sum
    

def main():
    data = getData() #data is a dataframe containing the dataset
    columnSet = data.keys().values #columnSet is an array containing the names of the columns
    
    targetColumnName = columnSet[-1]
    targetColumn = data[targetColumnName]
    targetEntropy = calculateEntropy(targetColumn)
    
    lastFeatureIndex = len(columnSet) - 1 #get index of last feature column
    
    #Loop through each column in the data set, calculating the IG for each
    for featureColumnNumber in range(1,lastFeatureIndex):
        featureColumnName = columnSet[featureColumnNumber]
    
        featureColumnFrame = data[featureColumnName].to_frame()
        targetColumnFrame = targetColumn.copy(deep=True).to_frame()
        reducedData = featureColumnFrame.join(targetColumnFrame) #reducedData is a dataframe with 1 feature column & 1 class column
        
        secondTerm = calculate2ndTerm(reducedData,featureColumnName)
        
        infoGain = targetEntropy - secondTerm
        
        infoGainRounded = round(infoGain,3)
        
        print("Information Gain Of Feature \"" + featureColumnName + "\" = " + str(infoGain))
        
        print("Rounded Information Gain = " + str(infoGainRounded) + "\n")
        
main()

Information Gain Of Feature "Outlook" = 0.24674981977443933
Rounded Information Gain = 0.247

Information Gain Of Feature "Temperature" = 0.02922256565895487
Rounded Information Gain = 0.029

Information Gain Of Feature "Humidity" = 0.15183550136234159
Rounded Information Gain = 0.152

Information Gain Of Feature "Wind" = 0.04812703040826949
Rounded Information Gain = 0.048

